In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
from business_duration import businessDuration
import holidays as hd
import business_duration as bd
from datetime import time

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format ='{:.2f}'.format
warnings.filterwarnings("ignore")

# Вспомогательные

In [3]:
# Сегодняшний день
today = dt.date.today()
# сейвер
archive_path_final_result =f"C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\Среднее кол-во часов отработки в ЦО\\Архив\\{today} Среднее время.xlsx"

## Даты отчёта

In [4]:
start_date = pd.to_datetime('2023-09-01')
end_date = pd.to_datetime('2023-09-24')

# Загрузка исходников

In [5]:
raw_complaints = pd.read_csv("C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\отчёт по ЦО\\xxx.csv",
                             encoding='utf-8', dtype={'Описание претензии': str}, sep=';')
raw_complaints_lines = pd.read_csv("C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\отчёт по ЦО\\yyy.csv",
                                   encoding='utf-8', dtype={'Описание претензии': str}, low_memory=False, sep=';')

VPR_office = pd.read_excel("C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\отчёт по ЦО\\ВПР\\ВПР офис-ЛЦ.xlsx")
office_dict = dict(zip(VPR_office['Текст'], VPR_office['Значение']))

VPR_class_71_51 = pd.read_excel("C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\отчёт по ЦО\\ВПР\\ВПР класс 71 51.xlsx")
class_dict = dict(zip(VPR_class_71_51['Текст'], VPR_class_71_51['Значение']))

VPR_executor_office = pd.read_excel("C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\отчёт по ЦО\\ВПР\\ВПР должность-офис2.xlsx")
executor_office = dict(zip(VPR_executor_office['Текст'], VPR_executor_office['Значение']))

VRP_office_RK = pd.read_excel("C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\отчёт по ЦО\\ВПР\\ВПР офис РК.xlsx")
executor_RK = dict(zip(VRP_office_RK['Текст'], VRP_office_RK['Значение']))


VRP_report_name = pd.read_excel("C:\\Users\\rassadin_mi\\Desktop\\автоматизация отчёт ЦО\\отчёт по ЦО\\ВПР\\ВПР отчёты.xlsx")
report_name = dict(zip(VRP_report_name['Этапы'], VRP_report_name['Наименование отчета']))

# Основная часть

In [6]:
# Преобразование форматов времени рекламаций
# Преобразование форматов времени
raw_complaints[['Дата запуска процесса', 
                'Дата завершения процесса']] = raw_complaints[[
                'Дата запуска процесса', 
                'Дата завершения процесса']].apply(pd.to_datetime)
raw_complaints['Дата запуска текущего этапа'] = pd.to_datetime(
    raw_complaints['Дата запуска текущего этапа'], format='%d.%m.%Y %H:%M:%S')

raw_complaints_lines[['Дата запуска процесса',
                      'Дата завершения процесса']] = raw_complaints_lines[[
                      'Дата запуска процесса',
                      'Дата завершения процесса']].apply(pd.to_datetime)
raw_complaints_lines['Дата запуска текущего этапа'] = pd.to_datetime(
    raw_complaints_lines['Дата запуска текущего этапа'], format='%d.%m.%Y %H:%M:%S')

In [7]:
# cортирую по убыванию количества часов , чтобы оставить бОльшие значения при дропе дубликатов
raw_complaints = raw_complaints.sort_values(['Количество часов отработки этапа процесса'], ascending=False)
raw_complaints.drop_duplicates(subset=['Id процесса','Текущий этап'],
                                                          keep='first', inplace=True)
complaints_no_duplicates = raw_complaints.drop(['Результат Возврата Товара','Результат Замены Довоза'], axis=1)

In [8]:
raw_complaints_lines = raw_complaints_lines.sort_values(['Количество часов отработки этапа процесса'], ascending=False)
raw_complaints_lines.drop_duplicates(subset=['Id процесса','Должность исполнителя этапа', 'Текущий этап'],
                                                          keep='first', inplace=True)
compliants_lines_no_duplicates = raw_complaints_lines.drop(['Решение','Описание','Комментарий','Производитель',
                                                'Код производителя','Код товара','Описание товара',
                                                'Класс 33','Цена за ед. измерения без НДС', 'Id процесса',
                                                            'Кол-во товара', 'Уход', 'Результат возврата/обмена/довоза',
                                                           'Ожидает ответа поставщика', 'ФИО сотр. допуст. ошибку',
                                                           'Должн. сотр. допуст. ошибку', 'Описание ошибки',
                                                            'Кто определил допуст. ошибку'], axis=1)
compliants_lines_no_duplicates = compliants_lines_no_duplicates.rename(columns={'Id родительского процесса':'Id процесса'})

In [9]:
len(compliants_lines_no_duplicates)

254023

In [10]:
len(complaints_no_duplicates)

264284

In [11]:
# сличяние построчного и рабочего
merged_compliants = pd.concat([compliants_lines_no_duplicates, complaints_no_duplicates], ignore_index=True)
len(merged_compliants)

518307

In [12]:
# Верный промежуток дат
merged_compliants = merged_compliants.query('`Дата завершения процесса` >= @start_date and `Дата завершения процесса` <= @end_date' )
# Верный этап
merged_compliants = merged_compliants.query('`Текущий этап` != ["Таймер","Этап автосогласования"]')

In [13]:
len(merged_compliants)

54440

In [14]:
def get_LC(row):
    if row['ОП по складу'] in office_dict:
        return office_dict[row['ОП по складу']]
       
def class71(row):
    if row['Класс 71'] in class_dict :
        return class_dict[row['Класс 71']]
    else:
        return 'не указано'
    
def class55(row):
    if row['Класс 55'] in class_dict:
        return class_dict[row['Класс 55']]    
    else:
        return 'не указано'
    
def get_office(row):
    if row['Должность исполнителя этапа'] in executor_office:
        return executor_office[row['Должность исполнителя этапа']]
    else:
        return row['ОП по складу']
    
def get_RK(row):
    if row['Офис исполнителя этапа'] in executor_RK:
        return executor_RK[row['Офис исполнителя этапа']]

def report(row):
    if row['Текущий этап'] in report_name:
        return report_name[row['Текущий этап']]
    else:
        return 'не указано'

In [15]:
merged_compliants['ЛЦ по складу'] = merged_compliants.apply(get_LC, axis=1)
merged_compliants['Класс 71.'] = merged_compliants.apply(class71, axis=1)
merged_compliants['Класс 55.'] = merged_compliants.apply(class55, axis=1)
merged_compliants['Офис исполнителя этапа'] = merged_compliants.apply(get_office, axis=1)
merged_compliants['РК'] = merged_compliants.apply(get_RK, axis=1)
merged_compliants['Отчёт'] = merged_compliants.apply(report, axis=1)

In [16]:
len(merged_compliants)

54440

In [17]:
merged_compliants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54440 entries, 5 to 518275
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   Id процесса                                54440 non-null  int64         
 1   РК по документу                            54440 non-null  object        
 2   ОП по складу                               54440 non-null  object        
 3   Номер документа                            54440 non-null  object        
 4   Код к/а                                    54440 non-null  int64         
 5   Название к/а                               54440 non-null  object        
 6   ФИО к/а                                    38708 non-null  object        
 7   Телефон к/а                                33576 non-null  object        
 8   Класс 71                                   54411 non-null  object        
 9   Класс 72        

In [18]:
merged_compliants['Дата завершения текущего этапа'] = merged_compliants[
    'Дата запуска текущего этапа'] + pd.to_timedelta(merged_compliants[
    'Количество часов отработки этапа процесса'], unit='h')

In [19]:
# список праздников
holidays_2023 = {datetime(2023,1,1).date():"New Year Holidays",
                 datetime(2023,1,2).date():"New Year Holidays",
                 datetime(2023,1,3).date():"New Year Holidays",
                 datetime(2018,1,4).date():"New Year Holidays",
                 datetime(2018,1,5).date():"New Year Holidays",
                 datetime(2023,1,6).date():"New Year Holidays",
                 datetime(2023,1,8).date():"New Year Holidays",
                 datetime(2023,1,7).date():"Christmas",
                 datetime(2023,2,23).date():"Motherland protector`s day",
                 datetime(2023,2,24).date():"Motherland protector`s day",
                 datetime(2023,3,8).date():"Woomen day",
                 datetime(2023,5,1).date():"Spring and labor day",
                 datetime(2023,5,9).date():"Victory day",
                 datetime(2023,5,8).date():"Victory day",
                 datetime(2023,6,12).date():"The day of Russia",
                 datetime(2023,11,4).date():"Day of i dont know whats is it",
                }



# подсчёт рабочих часов между двумя датами
def workin_hours(data):
    starttime=time(9,0,0)
    endtime=time(18,0,0)
    unit='hour'
    # сюда ещё нужно добавить holidays - атрибут для 
    data['Количество рабочих часов'] = ''
    for index, row in data.iterrows():
        data.at[index, 'Количество рабочих часов'] = businessDuration(data.at[index, 'Дата запуска текущего этапа'],
                                                               data.at[index, 'Дата завершения текущего этапа'],
                                                               starttime,endtime,unit=unit, 
                                                                holidaylist=holidays_2023)
    return data

workin_hours(merged_compliants)
merged_compliants.head(1)

,Id процесса,РК по документу,ОП по складу,Номер документа,Код к/а,Название к/а,ФИО к/а,Телефон к/а,Класс 71,Класс 72,Менеджер из документа,Должность менеджера из документа,Класс 55,РК исполнителя этапа,Должность исполнителя этапа,Описание претензии,Исполнитель текущего этапа,Текущий этап,Дата запуска процесса,Дата завершения процесса,Количество дней работы процесса,Дата запуска текущего этапа,Количество часов отработки этапа процесса,ЛЦ по складу,Класс 71.,Класс 55.,Офис исполнителя этапа,РК,Отчёт,Дата завершения текущего этапа,Количество рабочих часов
5,2372342,"Директор по продажам 2 РЦ Урал,Директор по про...",Нижний Тагил ОП1,509/138822-1,1219011,ЭЛЕКТРОМОНТАЖ ПКФ,NaN,NaN,0d4,~A,Саканцева Татьяна,Сотрудник ОП1 Н.Тагил,Э,Инженер по работе с рекламациями ЛЦ Екатеринбург,Инженер по работе с рекламациями ЛЦ Екатеринбург,Бракованная или некомплектная продукция,Говейлер Ирина,Ожидание ответа поставщика МРР (товар отправле...,2023-03-27,2023-09-06,163,2023-04-12 15:00:37,3526,Урал,Электромонтажные организации,менеджер по работе со строительными и электром...,ЛЦ Екатеринбург,УК,этап зависит от поставика,2023-09-06 13:00:37,907.00


In [20]:
merged_compliants['Количество рабочих часов'].sum()

336698.4077777802

In [21]:
merged_compliants.sort_values(['Количество рабочих часов'], ascending=False).head(1)
merged_compliants['маркер этапа'] = 1
merged_compliants.head(1)

,Id процесса,РК по документу,ОП по складу,Номер документа,Код к/а,Название к/а,ФИО к/а,Телефон к/а,Класс 71,Класс 72,Менеджер из документа,Должность менеджера из документа,Класс 55,РК исполнителя этапа,Должность исполнителя этапа,Описание претензии,Исполнитель текущего этапа,Текущий этап,Дата запуска процесса,Дата завершения процесса,Количество дней работы процесса,Дата запуска текущего этапа,Количество часов отработки этапа процесса,ЛЦ по складу,Класс 71.,Класс 55.,Офис исполнителя этапа,РК,Отчёт,Дата завершения текущего этапа,Количество рабочих часов,маркер этапа
5,2372342,"Директор по продажам 2 РЦ Урал,Директор по про...",Нижний Тагил ОП1,509/138822-1,1219011,ЭЛЕКТРОМОНТАЖ ПКФ,NaN,NaN,0d4,~A,Саканцева Татьяна,Сотрудник ОП1 Н.Тагил,Э,Инженер по работе с рекламациями ЛЦ Екатеринбург,Инженер по работе с рекламациями ЛЦ Екатеринбург,Бракованная или некомплектная продукция,Говейлер Ирина,Ожидание ответа поставщика МРР (товар отправле...,2023-03-27,2023-09-06,163,2023-04-12 15:00:37,3526,Урал,Электромонтажные организации,менеджер по работе со строительными и электром...,ЛЦ Екатеринбург,УК,этап зависит от поставика,2023-09-06 13:00:37,907.00,1


In [22]:
compliants_order = ['Id процесса','РК по документу','ОП по складу','ЛЦ по складу',
                    'Номер документа', 'Код к/а', 'Название к/а', 'ФИО к/а', 'Телефон к/а', 'Класс 71', 'Класс 71.',
                    'Класс 72', 'Менеджер из документа', 'Должность менеджера из документа', 'ЛЦ по складу',
                    'Класс 55', 'Класс 55.', 'РК исполнителя этапа', 'Должность исполнителя этапа',
                    'Офис исполнителя этапа', 'РК', 'Описание претензии', 'Исполнитель текущего этапа',
                    'Текущий этап', 'Дата запуска процесса', 'Дата завершения процесса',
                    'Количество дней работы процесса', 'Дата запуска текущего этапа', 
                    'Дата завершения текущего этапа', 'Количество часов отработки этапа процесса',
                    'маркер этапа', 'Количество рабочих часов', 'Отчёт']

merged_compliants = merged_compliants.reindex(columns=compliants_order)

## Среднее время МОПП, МООК, АП и КУ РЦ

In [23]:
average_time_sales = merged_compliants.query(
    '`Отчёт` == "Отчет по среднему времени отработки этапов в ЦО МОПП, МООК, АП и КУ РЦ."')

## Среднее время ЦПП

In [24]:
average_time_SSC = merged_compliants.query(
    '`Офис исполнителя этапа` == "ЦПП"')

## УЛ РЦ

In [25]:
average_time_LM1 =  merged_compliants.query(
    '`Отчёт` == "Отчет по среднему времени отработки этапов в ЦО сотрудниками УЛ РЦ"')
GZIK = ['Принять решение После допроверки НООК/Зав.ВТЦ/ГЗиК', 'Проверить наличие Брака НООК/Зав.ВТЦ/ГЗиК']
LC = ['ЛЦ Воронеж','ЛЦ Екатеринбург', 'ЛЦ Казань','ЛЦ Котельники','ЛЦ МЯ', 'ЛЦ Новосибирск', 'ЛЦ Ростов',
      'ЛЦ Самара','ЛЦ Урал', 'ЛЦ Шушары']
average_time_LM2 = merged_compliants.query(
    '`Отчёт` == "НООК" and `Текущий этап` == @GZIK and `Офис исполнителя этапа` == @LC')
average_time_LM = pd.concat([average_time_LM1, average_time_LM2], ignore_index=True)

In [26]:
average_time_LM2['Офис исполнителя этапа'].unique()

array(['ЛЦ Шушары', 'ЛЦ Самара', 'ЛЦ Казань', 'ЛЦ Екатеринбург',
       'ЛЦ Котельники', 'ЛЦ Ростов', 'ЛЦ МЯ', 'ЛЦ Воронеж'], dtype=object)

## Сохранение

In [27]:
# Создайте объект ExcelWriter для записи данных в файл Excel
writer = pd.ExcelWriter(archive_path_final_result)
# Запишите DataFrame в файл Excel
# Запишите DataFrame в файл Excel
average_time_SSC.to_excel(writer, sheet_name=f'ЦПП', index=False)

# Сохранение таблицы complianses_in_process_transposed на 1 лист
average_time_sales.to_excel(writer, sheet_name='Продажи', index=False)

# Сохранение таблицы complianses_in_process_2d_transposed на 2 лист
average_time_LM.to_excel(writer, sheet_name='Логисты', index=False)

# Сохранение таблицы complianses_in_process_2d_transposed на 2 лист
merged_compliants.to_excel(writer, sheet_name='общий исходник', index=False)

# Сохраните файл Excel
writer.save()